<a href="https://colab.research.google.com/github/AMerrington/sense-hackathon/blob/alex/SENSE_CDT_Practical_Session_Template.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Automated Sentinel-1 Ice, Water, Land Segmentation Challenge



This notebook is intended as a template only, to help guide through the training process. Feel free to use as little or as much of it as you like.

For the purposes of the template, we will assume a *classification* approach, which involves sub-sampling small images from the Sentinel-1 images. There will be notes where code should be adjusted for a *segmentation* approach.

### Mounting Google Drive


In [1]:
from google.colab import drive
drive.mount('/content/drive')

# Change the directory to the google drive
%cd /content/drive/My\ Drive/

import sys
sys.path.append('/content/drive/MyDrive/')

Mounted at /content/drive
/content/drive/My Drive


In [2]:
!pip install xarray
!pip install rasterio
!pip install geopandas

     |████████████████████████████████| 19.1MB 45.1MB/s 
     |████████████████████████████████| 972kB 11.9MB/s 
     |████████████████████████████████| 6.5MB 33.3MB/s 
     |████████████████████████████████| 14.8MB 305kB/s 


### Dataset preparation - (1) sub-sampling

Sample patches from each TIF image, and find the corresponding label using the Shapefiles. Save each image with a unique ID save in the directory **SAMPLING_DIR**. Save the corresponding meta data in the following format (this could be a CSV file, NumPy array, or some other format), in the directory **META_DIR**:


```
image_id, x, y, label
```

Set the label value as one of "L", "W", "I" as specified in the Shapefiles.

To make it easier to patch the final segmentation back together, it is suggested to use the (x, y) pixel coordinates of the patch, rather than the spatial coordinates.

In [ ]:
tile.values.shape

(152, 155, 3)

In [ ]:
"""
Importing the S1 images
"""
import os
import numpy as np
import xarray as xr
from write_tiff_funcs_DTM import create_geoTrans
from write_tiff_funcs_DTM import check_array_orientation
from write_tiff_funcs_DTM import write_xarray_to_GeoTiff

# Setting directories
SAMPLING_DIR = "/content/drive/MyDrive/Sentinel geotiffs/tiff_samples"
META_DIR = "/content/drive/MyDrive/Sentinel geotiffs/tiff_sample_metadata"
S1_tiffs= "/content/drive/MyDrive/Sentinel geotiffs"

# Importing S1 tiffs.
for filename in os.listdir(S1_tiffs):
    # Opening tiffs as an xarray.
    raster= xr.open_rasterio('%s/%s' % (S1_tiffs, filename))
    n_bands= raster.values.shape[0]
    
    # THIS IS NOT REFLECTANCE but does rescale (normalise bands) from 0 to 1.
    #img= exposure.rescale_intensity(raster)
    
    # apply NaN values to empty cells in multispec image.
    for ii in range(n_bands):
        raster.values= raster.values.astype('float')
        raster.values[raster.values==raster.nodatavals[ii]]= np.nan
        #raster= raster.rio.reproject('EPSG:27700') # reprojecting to British National Grid for use in GIS.
    
    raster= raster.transpose('y','x','band') 

    # Tiling this S1 tiff.
    img_shape = raster.values.shape
    num_tiles= 1000
    x_size= img_shape[1]/num_tiles
    y_size= img_shape[0]/num_tiles

    # xarray
    j= (0, 0)
    offset = (int(y_size), int(x_size))
    segments= np.zeros(raster.values.shape[0:2])
    
    x_coords= raster.coords['x'].values
    y_coords= raster.coords['y'].values

    for i in range(1, num_tiles + 1): 
            for ii in range(1, num_tiles + 1): 
                tile = raster.sel(x=slice(x_coords[j[0]], x_coords[offset[1]*i-1]), y=slice(y_coords[j[1]], y_coords[offset[0]*ii-1]))
                # Writing out the tiled tiff
                geoTrans= create_geoTrans(tile, x_name= 'x', y_name= 'y')
                check_array_orientation(tile,geoTrans,north_up=True)
                write_xarray_to_GeoTiff(tile, SAMPLING_DIR, north_up=True)
                # Moving the offset to the next column (row keeps static).
                j= (j[0], offset[1]*ii)
            # Moving to the next row and resetting the column offsets.
            j= (offset[0]*i, 0)
            print('Tile segmented: '+ str(i) +' of '+ str(num_tiles))

    


ValueError: ignored

Some helpful code: reading in a single Sentinel-1 image and the corresponding Shapefile.

In [ ]:
# the directory containing all shapefiles - i.e., the location of sea_ice/ 
SHAPEFILE_DIR = "/content/drive/MyDrive/EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice" 


shapefile = SHAPEFILE_DIR + "" # full name of .shp file

# extract the shape ID, for example, 20180116T075430
shp_id = shapefile.split("_")[-1][:-4].upper()

# locate the corresponding Sentinel-1 image based on the ID
# this should only return 1 match, which you can confirm
tiff_file = [g for g in tiff_files if shp_id in g]
tiff_file = tiff_file[0]

Feel free to use other Python packages; but as an example, here we use **geopandas** to read in the Shapefile, and **rasterio** to read the GeoTIFF.

In [ ]:
shape_data = gpd.read_file(SHAPEFILE_DIR + shapefile)

shape_data.head()

In [ ]:
# directory containing all GeoTIFF files
TIFF_DIR = ""

tif_img = rasterio.open(TIFF_DIR + tiff_file)

In [5]:
import os
import matplotlib.pyplot as plt
import geopandas as gpd
import cv2
import rasterio
from shapely.geometry import Point
from os import listdir
from os.path import isfile, join
import numpy as np

# Setting directories
SAMPLING_DIR = "/content/drive/MyDrive/Sentinel geotiffs/png_samples_100x100/"
META_DIR = "/content/drive/MyDrive/Sentinel geotiffs/tiff_sample_metadata/"
TIFF_DIR= "/content/drive/MyDrive/Sentinel geotiffs/"
SHAPEFILE_DIR= "/content/drive/MyDrive/EE_Polar_Training_Dataset_v-1-0-0/Sea_Ice/"

# Set up training arguments and parse
max_tries= 1200
samples_per_label= 400



# one by one read in Shapefiles
shp_files = [f for f in listdir(SHAPEFILE_DIR) if isfile(join(SHAPEFILE_DIR, f)) and ".shp" in f]
tiff_files = [f for f in listdir(TIFF_DIR) if isfile(join(TIFF_DIR, f)) and ".tif" in f]

print("Shapefiles found:       ", len(shp_files))
print("GeoTIFF files found:    ", len(tiff_files))

# function to determine whether a spatial coordinate is in any of the listed shapes
def get_label(shape_data, spatial_coords):

	spa_x, spa_y = spatial_coords

	for i, poly in enumerate(shape_data['geometry']):
		if poly.contains(Point(spa_x, spa_y)):
			return shape_data['poly_type'][i]

	return None


# each row corresponds to a sub_image - image ix, (x, y) for top left corner, label
meta = []

idx = 7520  # unique sub-sample image idx

def is_quota_met(count_dict):
	for val in count_dict.values():
		if val < samples_per_label:
			return False
	return True


# Local testing constraint
# shp_files = shp_files[:2]
# K defines the size of the image in pixels (KxK).
K= 100
for shpfile in shp_files:
	tries = 0
	shp_id = shpfile.split("_")[-1][:-4].upper()
	shape_data = gpd.read_file(SHAPEFILE_DIR + shpfile)

	# reset the sample count dict per image
	sample_count = {
		"L": 0,
		"W": 0,
		"I": 0,
	}

	# read in associated GeoTIFF file
	tiff_file = [g for g in tiff_files if shp_id in g]
	print(tiff_file[0])
  
	if len(tiff_file):
		src = rasterio.open(TIFF_DIR + tiff_file[0])

		# get numpy version for sampling sub_image
		np_tiff = np.rollaxis(src.read(), 0, 3)
    
		while not is_quota_met(sample_count) and tries < max_tries:

			tries += 1

			# select random x, y in src.width, src.height
			x_pos = np.random.randint(0, src.width)
			y_pos = np.random.randint(0, src.height)
      
			spatial_coords = src.transform * (x_pos + int(K/2), y_pos + int(K/2))

			# is this position in a shape?
			label = get_label(shape_data, spatial_coords)
			if label and sample_count[label] < samples_per_label:
				sub_im = np_tiff[x_pos:(x_pos + K), y_pos:(y_pos + K)]

				if sub_im.shape == (K, K, 3):
					cv2.imwrite(SAMPLING_DIR + str(idx) + ".png", sub_im)
					meta.append([idx, x_pos, y_pos, label])
					idx += 1

					sample_count[label] += 1

	print("Number of tries:", tries)

print("")
print("Final idx:              ", idx)
print("Meta length:            ", len(meta))
np.save(META_DIR + "meta3.npy", np.array(meta))

Shapefiles found:        12
GeoTIFF files found:     25
S1B_EW_GRDM_1SDH_20180515T174633_20180515T174733_010935_01403A_A84D_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1A_EW_GRDM_1SDH_20180717T073809_20180717T073909_022831_0279B9_EBF1_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1B_EW_GRDM_1SDH_20181113T074529_20181113T074629_013583_019254_D382_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1B_EW_GRDM_1SDH_20180213T175444_20180213T175544_009608_011511_8266_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1A_EW_GRDM_1SDH_20180612T180423_20180612T180523_022327_026AB3_AC33_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1A_EW_GRDM_1SDH_20180116T075430_20180116T075530_020177_0226B9_9FE3_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1A_EW_GRDM_1SDH_20180313T181225_20180313T181325_021000_0240E1_8163_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200
S1A_EW_GRDM_1SDH_20180417T074606_20180417T074706_021504_0250C3_D211_Orb_Cal_Spk_TC_rgb_8bit.tif
Number of tries: 1200


FileNotFoundError: ignored

The shapes in the Shapefiles are **shapely** objects. We can also use the Python package **shapely** to check whether an x, y pixel coordinate position is in a given polyshape.

In [ ]:
from shapely.geometry import Point

x = 4000
y = 8000

point = Point(x, y)

# for example, specify the shape in the Shapefile
shape_id = 2

if shape_data['geometry'][shape_id].contains(point):
    print("Point", point, "is in shape", shape_id, "and has class", shape_data['poly_type'][shape_id])

In [ ]:
x=np.load('/content/drive/MyDrive/Sentinel geotiffs/tiff_sample_metadata/meta3.npy')
x

array([['7520', '10906', '6314', 'W'],
       ['7521', '9200', '8760', 'W'],
       ['7522', '9828', '2899', 'W'],
       ...,
       ['10015', '3615', '7571', 'I'],
       ['10016', '2659', '5567', 'L'],
       ['10017', '2205', '6378', 'L']], dtype='<U21')

Define a train/validation ratio. Patches and meta saved from the test TIF images should be stored in separate directories.

In [ ]:
TRAIN_SIZE = 0.7

# valid size = 1.0 - TRAIN_SIZE

Map the class category characters to integers.

In [ ]:
LABELS = {
	"L": 0,
	"W": 1,
	"I": 2,
}

The following is a Dataset class which reads in image data saved in the format described above.

In [ ]:
from torch.utils.data import Dataset
from torchvision import transforms

from PIL import Image


class PolarPatch(Dataset):
    def __init__(self, transform=None, split="train"):
        super(PolarPatch, self).__init__()

        assert split in ["train", "val"]
        
        # TODO: load in meta data, which should be of shape (3, N) - N being the number of samples
        meta = []

        train_dim = int(TRAIN_SIZE * len(meta))
        
        if split == "train":
            meta = meta[:train_dim]
        else:
            meta = meta[train_dim:]                   

        self.images = range(len(meta))
        self.coords = [(row[1], row[2]) for row in meta]

        # Targets in integer form for computing cross entropy
        self.targets = [LABELS[row[3]] for row in meta]
        self.transform = transform


    def __len__(self):
        return len(self.targets)

    def __getitem__(self, index):

        x = Image.open(SAMPLING_DIR + str(self.images[index]) + ".png") # change this file format if needed
        y = self.targets[index]
        coord = self.coords[index]

        if self.transform:
        	x = self.transform(x)

        return x, y, coord

An example data transform

In [ ]:
data_transform = transforms.Compose([
    # TODO: add whatever else you need - normalisation, augmentation, etc.
	transforms.ToTensor(),
])

### Dataset preparation - (2) data loaders

Now we can prepare the data loaders. Here is the example for the training set; you will also need the validation and test set.

In [ ]:
import torch

# TODO set this value based on your working environment
BATCH_SIZE = 128

train_set = PolarPatch(
    split='train',
    transform=data_transform
)

train_loader = torch.utils.data.DataLoader(
    train_set,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=2
)

### Model

You can use a custom model architecture, or copy one from literature. It is recommended to not build too deep of a network for the sake of training time.

In [ ]:
import torch.nn as nn


class PolarNet(nn.Module):
    def __init__(self, n_classes=3):
        super(PolarNet, self).__init__()

        self.features = nn.Sequential(
            # TODO: build your own architecture here; one conv layer and ReLU here as an example only
            nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
            nn.ReLU(inplace=True), 
        )

        self.classifier = nn.Sequential(
            # TODO: continue classifier section of architecture here for classification approach;
            # otherwise, remove and add in upscaling for a fully-convolutional segmentation approach 
            nn.Linear(4096, n_classes),
        )      

    def forward(self, x):
        # as an example; alter as needed depending on your architecture
        x = self.features(x)

        x = torch.flatten(x, 1)
        x = self.classifier(x)
        return x

### Training

An example of loading the model, setting a loss criteria and defining an optimizer.

In [ ]:
# Device configuration - defaults to CPU unless GPU is available on device
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

In [ ]:
from torch import optim

model = PolarNet().to(DEVICE)
criterion = nn.CrossEntropyLoss()

# Stochastic gradient descent - TODO: alter as needed
optimizer = optim.SGD(
	model.parameters(),
	lr=0.001,
	weight_decay=0.0005,
	momentum=0.9,
)

Train the model, batch by batch, for as many iterations as required to converge. You can use the validation set to determine automatically when to stop training.

### Evaluation

Evaluate patch-based accuracy on the test set; then using the test patch coordinates, piece together the segmentation prediction on the original TIF images.